# Creating feature dataframe

## Loading libraries and dataset

In [ ]:
# importing libraries
import pyreadr
import numpy as np
import pandas as pd
import datetime #used for date and time calculations
pd.options.mode.chained_assignment = None  # remove an annoying warning

# loading the dataset
data = pyreadr.read_r('/path/to/file.rds')
df=data[None]

## Add columns IS_PEAKTIME and IS_WEEKDAY

In [ ]:
# adding column with weekday (0 = monday, 1 = tuesday, ...)
df['WEEKDAY'] = df['TIME_REAL'].dt.dayofweek

# adding column with boolean is_weekday (True = weekday, False = weekend)
df["IS_WEEKDAY_BOOL"] = df["WEEKDAY"] <= 4

# convert boolean to an int
df["IS_WEEKDAY"] = df["IS_WEEKDAY_BOOL"].astype(int)

# variables that decide peaktimes
peak_m_s = datetime.datetime(1999, 11, 13, 6)
peak_m_e = datetime.datetime(1999, 11, 13, 9)
peak_e_s = datetime.datetime(1999, 11, 13, 16)
peak_e_e = datetime.datetime(1999, 11, 13, 19)

# add column with a boolean for peaktime
df['IS_PEAKTIME_BOOL'] = (((df['TIME_REAL'].dt.time < peak_m_e.time()) & (df['TIME_REAL'].dt.time >= peak_m_s.time())) | ((df['TIME_REAL'].dt.time < peak_e_e.time()) & (df['TIME_REAL'].dt.time >= peak_e_s.time())))

#convert the boolean to an integer
df['IS_PEAKTIME'] = df['IS_PEAKTIME_BOOL'].astype(int)

# remove columns
df = df.drop(["IS_PEAKTIME_BOOL"], axis=1) 

## Add columns PRECEDING_STATION and PRECEDING_DELAY

In [ ]:
df = df.reset_index(drop=True) #reset index to access rows in loops

preceding_station = [df.loc[0, 'STATION']]
preceding_delay = [df.loc[0, 'DELAY']]

for i in range (1, len(df)):
    preceding_station.append(df.loc[i-1, 'STATION'])
    preceding_delay.append(df.loc[i-1, 'DELAY'])
    
df['PRECEDING_STATION'] = preceding_station
df['PRECEDING_DELAY'] = preceding_delay

## Add column TRAVEL_TIME 

In [ ]:
# create a list with the travel times from preceding to actual station
time = [df.loc[0, 'TIME_PLANNED']]

for i in range(1,len(df)):
    time.append(df.loc[i-1, 'TIME_PLANNED'])
    
df['PRECEDING_TIME'] = time

df['TRAVEL_TIME_TIMEDELTA'] = df['TIME_PLANNED'] - df['PRECEDING_TIME']
df['TRAVEL_TIME'] = df['TRAVEL_TIME_TIMEDELTA'].dt.total_seconds() / 60

# remove columns
df = df.drop(["PRECEDING_TIME", "TRAVEL_TIME_TIMEDELTA"], axis=1)

## Reduce dataframe to trains in the corridor Chur - Zurich HB

In [ ]:
# list containing all stations in the corridor:
stations = ['ZÃ¼rich HB', 'ZÃ¼rich Wiedikon', 'ZÃ¼rich Enge', 'ZÃ¼rich Wollishofen', 'Kilchberg', 'RÃ¼schlikon', 'Thalwil', 'Oberrieden', 'Horgen', 'Au ZH', 'WÃ¤denswil', 'Richterswil', 'BÃ¤ch', 'Freienbach SBB', 'PfÃ¤ffikon SZ', 'Altendorf', 'Lachen', 'Siebnen-Wangen', 'SchÃ¼belbach-Buttikon', 'Reichenburg', 'Bilten', 'ZiegelbrÃ¼cke', 'MÃ¼hlehorn', 'Murg', 'Unterterzen', 'Mols', 'Walenstadt', 'Flums', 'Mels', 'Sargans', 'Bad Ragaz', 'Maienfeld', 'Landquart', 'Chur']

stations_df = pd.DataFrame(stations, columns = ['stations'])

#create a list of booleans whether the station is in the corridor or not
check_station = []
check_previous_station = []

for i in range (len(df)):
    check_station.append(df.loc[i, 'STATION'] in stations)
    check_previous_station.append(df.loc[i, 'PRECEDING_STATION'] in stations)
    
#append the list to the last column of the dataframe 
df['CHECK_STATION'] = check_station
df['CHECK_PRECEDING_STATION'] = check_previous_station

# create a dataframe just with trains in the corridor Chur - Zürich HB
df = df.loc[(df['CHECK_STATION'] == True) & (df['CHECK_PRECEDING_STATION'] == True)]

# remove columns
df = df.drop(["CHECK_STATION"], axis=1)
df = df.drop(["CHECK_PRECEDING_STATION"], axis=1)

# resetting the indexes to acces rows in the loop
df = df.reset_index(drop=True)

## Add column PRECEDING_DISTANCE

In [ ]:
# import distance matrix
dima = pd.read_csv('/path/to/file.csv', sep=',')

# convert dataframe to numpy array
dima = np.array(dima)

# name columns and rows
distance_matrix = pd.DataFrame(dima, columns = ['ZÃ¼rich HB', 'ZÃ¼rich Wiedikon', 'ZÃ¼rich Enge', 'ZÃ¼rich Wollishofen', 'Kilchberg', 'RÃ¼schlikon', 'Thalwil', 'Oberrieden', 'Horgen', 'Au ZH', 'WÃ¤denswil', 'Richterswil', 'BÃ¤ch', 'Freienbach SBB', 'PfÃ¤ffikon SZ', 'Altendorf', 'Lachen', 'Siebnen-Wangen', 'SchÃ¼belbach-Buttikon', 'Reichenburg', 'Bilten', 'ZiegelbrÃ¼cke', 'MÃ¼hlehorn', 'Murg', 'Unterterzen', 'Mols', 'Walenstadt', 'Flums', 'Mels', 'Sargans', 'Bad Ragaz', 'Maienfeld', 'Landquart', 'Chur'], 
                  index=['ZÃ¼rich HB', 'ZÃ¼rich Wiedikon', 'ZÃ¼rich Enge', 'ZÃ¼rich Wollishofen', 'Kilchberg', 'RÃ¼schlikon', 'Thalwil', 'Oberrieden', 'Horgen', 'Au ZH', 'WÃ¤denswil', 'Richterswil', 'BÃ¤ch', 'Freienbach SBB', 'PfÃ¤ffikon SZ', 'Altendorf', 'Lachen', 'Siebnen-Wangen', 'SchÃ¼belbach-Buttikon', 'Reichenburg', 'Bilten', 'ZiegelbrÃ¼cke', 'MÃ¼hlehorn', 'Murg', 'Unterterzen', 'Mols', 'Walenstadt', 'Flums', 'Mels', 'Sargans', 'Bad Ragaz', 'Maienfeld', 'Landquart', 'Chur']) 

# creating a list with all the distances between preceding and actual station
distance = []

for i in range(len(df)):
    distance.append(distance_matrix.loc[(df.loc[i,'STATION']), (df.loc[i, 'PRECEDING_STATION'])])

# add list with distances as new column in df
df['PRECEDING_DISTANCE'] = distance

## Export dataframe as csv

In [ ]:
#filter dataframe to containing only arrival events
df = df.loc[df['EVENT_TYPE'] == 'ARRIVAL'] 

# export as csv file for faster imports in other code files
df.to_csv (r'/path/to/file.csv', index = False, header=True)